## Data Quality labelling
#### BDA project report

Report completed by:
* Anna Panfil
* Igor Czudy
* Juras Lukaševičius

In [51]:
# Setting directory

import os
os.chdir('D:/Users/Vartotojas/Documents/GitHub/data_quality_labeler')

In [52]:
# Creating dataset

from dataset_creator import FakeDataset, MISSING_SYMBOLS

filename = 'dataset.csv'
OUTLIER_PERCENTAGE = 0.1
DUPLICATE_PERCENTAGE = 0.15
MISSING_PERCENTAGE = 0.1


dataset = FakeDataset(dataset_size = 100)\
        .add_dominated_string_column(dominated_percentage=0.9)\
        .add_mishmashed_case(mishmashed_percentage=0.1)\
        .add_outliers_above(outlier_percentage = OUTLIER_PERCENTAGE)\
        .add_duplicates(duplicate_percentage = DUPLICATE_PERCENTAGE)\
        .add_missing(missing_percentage = MISSING_PERCENTAGE)\
        .to_csv(filename)

In [53]:
# Reading data
import numpy as np
import pandas as pd

data = pd.read_csv(filename)
print(data.shape)
data.head()

(115, 8)


,name,surname,birthdate,results1,results2,category,email,gender
0,Samantha,Martinez,1918-09-14,85,0.33799399624313065,A,david70@example.com,NaN
1,Chelsey,Gillespie,1944-02-19,41,1.1028054198064805,C,chentammy@example.org,F
2,Tyler,Hall,2023-12-19,56,2.3695139681554642,A,aramirez@example.net,F
3,Andrew,Owens,1988-07-17,21,-0.6611189327249538,NaN,jhunt@example.org,M
4,Lauren,King,2012-05-09,NaN,1.1390247312700723,A,donald13@example.org,F


In [54]:
from collections import defaultdict
dataset_scores = defaultdict(lambda: 0)

### Checking if there are any missing values

In [55]:
data.replace(MISSING_SYMBOLS, np.nan, inplace=True)

In [56]:
data.isna().sum()

name         12
surname      12
birthdate    12
results1     12
results2     12
category     12
email        12
gender       12
dtype: int64

In [57]:
data = data._convert(numeric=True, datetime=True).convert_dtypes()

In [58]:
dataset_scores["missing_percentage"] = data.isna().sum().sum()/data.size
dataset_scores["most_missing_column"] = data.isna().sum().max()/data.shape[0]

In [59]:
assert(round(dataset_scores["missing_percentage"],2) == MISSING_PERCENTAGE)

#IDEA: We can check how easy it is to train the model to replace missing values (looking at the error).
For all columns with nan % > sth. Random forest.

## Check duplicates

In [60]:
dataset_scores["duplication_percentage"] = sum(data.duplicated())/ data.shape[0]

## Check outliers

For numerical values we use the same method as in box plot (outlier is more tham q3 + 1.5 IQR or less than q1 - 1.5 IQR)

In [61]:
numeric_cols = data.select_dtypes(include=['number']).columns
string_cols = data.select_dtypes(include=["string", "object"]).columns
outliers_nums = []

for col in numeric_cols:
    
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    
    iqr = q3-q1
    
    upper_bound = q3 + (1.5*iqr)
    lower_bound = q1 - (1.5*iqr)

    outliers_nums.append(np.sum((data[col] > upper_bound) | (data[col] < lower_bound)))

For string columns we look for rare values (less than 5% of the observations)

In [62]:
for col in string_cols:
    if not len(data[col].unique())/len(data[col]) > 0.5 and\
            (rare := data["category"].value_counts().min())/data.shape[0] < 0.05: # rare category
        outliers_nums.append(rare)

dataset_scores["outliers_percentage"] = sum(outliers_nums)/data.size
dataset_scores["most_outliers_column"] = max(outliers_nums)/data.shape[0]

We also look for dominant values (more than 80% of the observations in column) and columns with unique values (eg. id, email), which may be not useful in further predictions.

In [63]:
for col in data.columns:  
    if len(data[col].unique())/len(data[col]) > 0.5: # column with rather unique values
        dataset_scores["unique_columns"] += 1
    if data[col].value_counts().max()/data.shape[0] > 0.8: # dominant category
        dataset_scores["dominated_columns"] += 1
        
dataset_scores["dominated_columns"] /= len(data.columns)
dataset_scores["unique_columns"] /= len(data.columns)

## Check mishmashed formats

In [64]:
mishmashed_cases = []
for col in string_cols:
    unique_in_data = len(data["category"].unique())
    truly_unique = len(data["category"].map(lambda x: x.lower() if not pd.isna(x) else x).unique())

    mishmashed_cases.append((unique_in_data - truly_unique)/truly_unique)

dataset_scores["max_mishmashed_case"] = max(mishmashed_cases)

#IDEAS: other measurements about this

Other ideas: correlation, not good dates, rules provided by user

## Aggregate scores

In [65]:
dataset_scores

defaultdict(<function __main__.<lambda>()>,
            {'missing_percentage': 0.10434782608695652,
             'most_missing_column': 0.10434782608695652,
             'duplication_percentage': 0.017391304347826087,
             'outliers_percentage': 0.01847826086956522,
             'most_outliers_column': 0.09565217391304348,
             'unique_columns': 0.75,
             'dominated_columns': 0.125,
             'max_mishmashed_case': 0.75})

In [66]:
weights = {
    "missing_percentage": 10,   # many missing values is difficult to handle
    "most_missing_column": 2,   # if 1 we had a column with huge amount of missing values, we'd have to drop it
    "duplication_percentage": 4,# many duplicates means less data
    "outliers_percentage": 2,   # outliers may be removed or cause problems with predictions
    "most_outliers_column": 1,
    "unique_columns": 5,        # if all columns are unique, we can't do much with it
    "dominated_columns": 3,     # if a column has one dominant category, it may be not very useful
    "max_mishmashed_case": 1    # our data may be dirty and require a lot of cleaning
}

assert weights.keys() == dataset_scores.keys()

In [67]:
final_score = 0
for name, score in dataset_scores.items():
    final_score += score * weights[name]

final_score /= sum(weights.values())
final_score = 1 - final_score # 1 is the best score, 0 – the worst
final_score

dataset_scores["dataset_quality_score"] = final_score

### Checking for formatting

In [68]:
# Here we check if columns only with numbers. If so, they are formatted as float64
# IMPORTANT! If the column is only numerical and an identifier, its name must be
# listed below. Otherwise, it will be added into outlier calculation.

categorical_variables = ['name']

In [69]:
string_columns = data.select_dtypes(include='object').columns

# Convert string columns to numeric if they contain only numbers or 'NaN'
for col in string_columns:
    try:
        # Skip conversion for columns in list A
        if col in categorical_variables:
            continue
            
        # Check if there are any numbers in the column
        if pd.to_numeric(data[col], errors='coerce').notna().any():
            data[col] = pd.to_numeric(data[col], errors='coerce')
        else:
            # If no numbers found, keep the column as an object
            data[col] = data[col].astype('object')
    except ValueError:
        print(f"Unable to convert column '{col}' to numeric.")

# Check the result
print("Updated DataFrame:")
print(data)

Updated DataFrame:
            name    surname   birthdate  results1  results2 category  \
0       Samantha   Martinez  1918-09-14        85  0.337994        A   
1        Chelsey  Gillespie  1944-02-19        41  1.102805        C   
2          Tyler       Hall  2023-12-19        56  2.369514        A   
3         Andrew      Owens  1988-07-17        21 -0.661119     <NA>   
4         Lauren       King  2012-05-09      <NA>  1.139025        A   
..           ...        ...         ...       ...       ...      ...   
110       Thomas      Adams  1943-06-10         0 -0.302413        A   
111       Angela      Foley  1936-01-16        20 -0.555290        C   
112        Linda     Lester  1995-05-27        79 -0.069751        B   
113  Christopher     Harris  2013-05-15        25 -0.658892        C   
114          Jon       Rich  1979-12-22        68  1.367867        A   

                            email gender  
0             david70@example.com   <NA>  
1           chentammy@example.

### Checking for outliers

In [70]:
# First we select the numeric columns in the data frame

numeric_columns = data.select_dtypes(include=['number']).columns
numeric_columns

Index(['results1', 'results2'], dtype='object')

In [71]:
# Function for outlier detection using descriptive statistics (quantiles)

def identify_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return column[(column < lower_bound) | (column > upper_bound)]

In [72]:
# Identify outliers in each numeric column

outliers_dict = {col: identify_outliers(data[col]) for col in numeric_columns}

In [73]:
numerical_values = len(data[list(outliers_dict.keys())])
outlier_count = 0

for col, outliers in outliers_dict.items():
        print(f"Potential number of outliers in {col}:")
        print(len(outliers))
        print("\n")
        outlier_count = len(outliers) + outlier_count

if numerical_values == 0:
    dataset_scores["outlier_percentage"] = 0
else:
    dataset_scores["outlier_percentage"] = outlier_count/numerical_values

Potential number of outliers in results1:
11


Potential number of outliers in results2:
0




### Duplicate detection (Lower case/higher case)

In [111]:
# Example data
info = {'Name': ['John', 'Mary', 'Bob', 'Bob', 'john', 'mary', 'BOB', 'Thomas', 'thomas', 'THOMAS', 'Thomas', 'Bob'], 
       'Day': ['noOn', 'nOon', 'Noon', 'noon', 'NOon', 'noon', 'noON', 'noon', 'nOON', 'NOON', 'noon', 'NoOn']}

# Create DataFrame
df = pd.DataFrame(info)

# Display the DataFrame
print("Original DataFrame:")
print(df)

Original DataFrame:
      Name   Day
0     John  noOn
1     Mary  nOon
2      Bob  Noon
3      Bob  noon
4     john  NOon
5     mary  noon
6      BOB  noON
7   Thomas  noon
8   thomas  nOON
9   THOMAS  NOON
10  Thomas  noon
11     Bob  NoOn


In [112]:
n_unique_val = 0
n_variants = 0

def get_case_duplicates(column):
    """
    Get the list of similar values (ignoring NaN) that only differ by capitalization in a column.
    
    Parameters:
    - column: pandas Series, the column to check
    
    Returns:
    - has_duplicates: True if case-insensitive duplicates are found, False otherwise
    - count_duplicates: count of similar values differing only by capitalization
    - duplicate_values: list of similar values differing only by capitalization
    """
    lowercased_values = column.dropna().astype(str).str.lower()
    
    def are_truly_case_duplicates(val1, val2):
        return val1 != val2 and val1.lower() == val2.lower()
    
    duplicated_mask = lowercased_values.duplicated(keep=False)
    has_duplicates = duplicated_mask.any()
    
    count_duplicates = 0
    duplicate_values = []
    
    for val in lowercased_values[duplicated_mask].unique():
        similar_values = lowercased_values[lowercased_values == val].index.tolist()
        if len(similar_values) > 1 and not any(are_truly_case_duplicates(lowercased_values[i], lowercased_values[j]) for i in similar_values for j in similar_values if i != j):
            count_duplicates += 1
            duplicate_values.extend(similar_values)
    
    return has_duplicates, count_duplicates, column[duplicate_values].tolist()

object_columns = data.select_dtypes(include='object').columns

for column_name in object_columns:
    has_duplicates, count_duplicates, duplicate_values = get_case_duplicates(data[column_name])

    if has_duplicates:
        print(f"The column '{column_name}' has {count_duplicates} truly similar values differing only by capitalization.")
        print(f"List of case-sensitive duplicate values: {duplicate_values}")
        unique_values = set(duplicate_values)
        unique_list = list(unique_values)
        print('\n')
        print(f"Unique values: {unique_list}")
        
        n_unique_val = n_unique_val + count_duplicates # number of actually unique values
        n_variants = n_variants + len(unique_list) # number of these unique value variants (with different capitalization)
        
    else:
        print(f"The column '{column_name}' does not have truly similar values differing only by capitalization.")
    print("\n")  # Add a separator for better readability

In [113]:
if n_variants == 0:
    dataset_scores["duplicate_proportion"] = 0
else:
    dataset_scores["duplicate_proportion"] = 1 - n_unique_val/n_variants  # the lower the score, the less duplicates there are - 0 is a better score

In [114]:
dataset_scores["duplicate_proportion"]

0

### Check documentation

This badge mainly shows that there is evidence of documentation, but can't be certain of its validity.

In [130]:
doc_dest = 'D:/Users/Vartotojas/Documents/GitHub/data_quality_labeler/documentation.txt' # destination of documentation

with open(doc_dest, 'r') as file:
    content = file.read().lower() # Lowercase to avoid inconsistensies

In [131]:
search_strings = []

variable_definitions_synonyms = [[
    "variable definitions",
    "variable declarations",
    "variable assignments",
    "declaration of variables",
    "defining variables",
    "variable initialization",
    "variable setup",
    "variable specification",
    "variable establishment",
    "variable creation",
    "variable naming",
    "setting variables",
    "variable instantiation",
    "variable initialization",
    "defining data elements",
    "variable configuration"
]]

variable_formatting_constraints_synonyms = [[
    "variable formatting constraints",
    "formatting restrictions for variables",
    "constraints on variable formatting",
    "variable format limitations",
    "rules for formatting variables",
    "constraints for variable presentation",
    "formatting guidelines for variables",
    "variable presentation constraints",
    "variable formatting rules",
    "limitations on variable format",
    "formatting specifications for variables",
    "variable format constraints",
    "variable styling constraints",
    "variable appearance rules"
]]

author_synonyms = [[
    "author",
    "creator",
    "writer",
    "originator",
    "contributor",
    "developer",
    "composer",
    "maker",
    "designer",
    "architect",
    "producer",
    "editor",
    "compiler",
    "engineer",
    "craftsman",
    "builder"
]]

date_synonyms = [[
    "date",
    "time",
    "timestamp",
    "day",
    "moment",
    "occasion",
    "event time",
    "calendar date",
    "point in time",
    "temporal",
    "chronology",
    "timeframe",
    "period",
    "date and time",
    "timing",
    "schedule"
]]

place_synonyms = [[
    "place",
    "location",
    "spot",
    "site",
    "position",
    "area",
    "region",
    "locale",
    "venue",
    "setting",
    "point",
    "site",
    "geographic location",
    "positioning",
    "spot",
    "address"
]]

allowable_ranges_synonyms = [[
    "allowable ranges",
    "permissible intervals",
    "acceptable spans",
    "allowed boundaries",
    "permitted scopes",
    "acceptable extent",
    "valid ranges",
    "tolerable limits",
    "authorized variations",
    "acceptable thresholds",
    "sanctioned intervals",
    "legitimate scopes",
    "admissible spans",
    "approved boundaries",
    "permissible ranges",
    "allowed margins"
]]

rules_synonyms = [[
    "rules",
    "guidelines",
    "instructions",
    "regulations",
    "policies",
    "procedures",
    "directives",
    "standards",
    "protocols",
    "requirements",
    "specifications",
    "criteria",
    "conditions",
    "principles",
    "laws",
    "norms"
]]

In [132]:
search_strings = variable_definitions_synonyms + variable_formatting_constraints_synonyms + rules_synonyms + author_synonyms + date_synonyms + place_synonyms + allowable_ranges_synonyms

In [133]:
context = 0

for search_string in search_strings:
    for i in search_string:
        if i in content:
            print(f"Found string: {i}")
            context = context + 1
            break

Found string: variable definitions
Found string: rules
Found string: author
Found string: moment
Found string: place
Found string: allowable ranges


In [134]:
dataset_scores["documentation_detail"] = context / len(search_strings) # 1 is the best score, 0 is the worst
dataset_scores["documentation_detail"]

0.8571428571428571

### Creating badges

In [100]:
# save obtained scores to json
import json
 
filename="./badge_data.json"
json_object = json.dumps(dataset_scores, indent=4)
 
with open(filename, "w") as outfile:
    outfile.write(json_object)

In [101]:
repo_url = "https://github.com/annapanfil/data_quality_labeler" #todo: get dinamically

ownername = repo_url.split("/")[3]
repo_name = repo_url.split("/")[4]
branch = "main"


print("To add badges paste this to your readme.md file:")
for badge in dataset_scores.keys():
    print(f"![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2F{ownername}%2F{repo_name}%2F{branch}%2F{filename}&query=%24.{badge}&label={badge})")


To add badges paste this to your readme.md file:
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2Fannapanfil%2Fdata_quality_labeler%2Fmain%2F./badge_data.json&query=%24.missing_percentage&label=missing_percentage)
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2Fannapanfil%2Fdata_quality_labeler%2Fmain%2F./badge_data.json&query=%24.most_missing_column&label=most_missing_column)
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2Fannapanfil%2Fdata_quality_labeler%2Fmain%2F./badge_data.json&query=%24.duplication_percentage&label=duplication_percentage)
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2Fannapanfil%2Fdata_quality_labeler%2Fmain%2F./badge_data.json&query=%24.outliers_percentage&label=outliers_percentage)
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw

In [ ]:
# Example data
info = {'Name': ['John', 'Mary', 'Bob', 'Bob', 'john', 'mary', 'BOB', 'Thomas', 'thomas', 'THOMAS', 'Thomas', 'Bob'], 
       'Day': ['noOn', 'nOon', 'Noon', 'noon', 'NOon', 'noon', 'noON', 'noon', 'nOON', 'NOON', 'noon', 'NoOn']}

# Create DataFrame
df = pd.DataFrame(info)

# Display the DataFrame
print("Original DataFrame:")
print(df)